In [3]:
import pandas as pd
import numpy as np

In [38]:
taxon_df = pd.read_csv("données/taxon.csv")
taxon_denombre_df = pd.read_csv("données/taxon_denombre.csv")
liste_df = pd.read_csv("données/liste_faunistique_floristique.csv")
stations_df = pd.read_csv("données/station.csv")
physicochimie_df = pd.read_csv("données/physicochimie.csv")
alteration_df = pd.read_csv("données/classe_alteration_seq_eau.csv")

In [5]:
physicochimie_df.sort_values("stn_id")

,stn_id,date_year,valeur,parameter,parameter_name,unite,unite_name
0,398944,2007,10.825000,1301,Température de l'Eau,27,degré Celsius
90,398944,2012,5.333333,1348,Silice,162,milligramme par litre
89,398944,2011,4.991667,1348,Silice,162,milligramme par litre
88,398944,2010,5.357200,1348,Silice,162,milligramme par litre
87,398944,2009,4.991667,1348,Silice,162,milligramme par litre
...,...,...,...,...,...,...,...
187356,421014,2007,97.333333,1312,Taux de saturation en oxygène,243,pourcentage
187355,421014,2012,10.766667,1311,Oxygène dissous,175,milligramme d'oxygène par litre
187354,421014,2011,10.566667,1311,Oxygène dissous,175,milligramme d'oxygène par litre
187367,421014,2012,0.733333,1313,Demande Biochimique en oxygène en 5 jours (D.B...,175,milligramme d'oxygène par litre


On récupère la liste complète des paramètres mesurés sur au moins un taxon

In [6]:
taxon_denombre_df["critere_taxon"].unique().tolist()

["Nombre exact d'individus",
 'Longueur Minimale totale exprimée en mm',
 'Poids individuel exprimé en g',
 'Longueur Individuelle totale exprimée en mm',
 "Poids total de l'ensemble des individus appartenant à un même taxon exprimé en g (issu de caluls d'estimation ou de mesures précises)",
 'Longueur Maximale totale exprimée en mm',
 'Abondance Relative (%)',
 "Poids individuel exprimé en g à partir d'une estimation réalisée sur d'autres points de contrôle de la DR",
 "Poids total de l'ensemble des individus appartenant à un même taxon exprimé en g à partir d'une estimation réalisée sur d'autres points de contrôle de la DR",
 "Poids individuel exprimé en g partir d'un autre cas",
 'Longueur Individuelle à la fourche exprimée en mm',
 "Poids individuel exprimé en g dont la méthode d'estimation n'est pas déterminée",
 'Longueur Minimale à la fourche exprimée en mm',
 "Poids total de l'ensemble des individus appartenant à un même taxon exprimé en g dont la méthode d'estimation n'est pas

On s'intéresse à l'évolution du nombre d'individus, on isole ainsi les lignes s'intéressant à cette propriété

In [7]:
critere_nombre_individus = taxon_denombre_df[taxon_denombre_df["critere_taxon"] == "Nombre exact d'individus"]\
    ["critere_taxon_id"]\
    .unique()\
    .tolist()
critere_nombre_individus

[1]

On vérifie la cohérence entre l'id du critère et sa description (l'autre sens a déjà été vérifié par la selection précédente)

In [8]:
df = taxon_denombre_df[taxon_denombre_df["critere_taxon_id"].isin(critere_nombre_individus)]
df["critere_taxon"].unique()

array(["Nombre exact d'individus"], dtype=object)

On ne s'intéresse qu'à un unique critère. On a vérifié la cohérence de notre séléction : les colonnes propres aux critères sont inutiles

Ensuite, on remarque qu'une seule mesure est bien effectuée par taxon par liste ff

In [9]:
individus_taxons = df.drop(columns=["critere_taxon", "critere_taxon_id"])

individus_taxons\
    .groupby(["taxon_id", 'liste_ff_id'])\
    .count()\
    ["valeur"]\
    .unique()

array([1])

In [10]:
mesure_par_taxon = individus_taxons\
    .drop(columns=["valeur"])\
    .groupby(["taxon_id"])\
    .count()\
    ["liste_ff_id"]

taxons_to_keep = mesure_par_taxon[mesure_par_taxon > 21000]\
    .index\
    .tolist()
taxons_to_keep

[212, 364, 807, 906, 933]

In [34]:
test_indiv = individus_taxons[individus_taxons["taxon_id"].isin(taxons_to_keep)]
test_indiv

,liste_ff_id,taxon_id,valeur
3,3551258,906,120.0
12,3551264,807,20.0
16,3551362,933,8.0
23,3551360,212,3.0
26,3551363,807,35.0
...,...,...,...
1569103,3625796,807,13.0
1569104,3625797,364,70.0
1569112,3625795,364,47.0
1569127,3625797,212,4.0


In [12]:
liste_df

,id,date_year,date_prel,liste_faunistique_floristique,facies_substrat,facies_vitesse,facies_profondeur
0,398943,2010,2010-08-11 00:00:00.000,3541596,"Pierres, galets",Vitesse < 5 cm/s,NaN
1,398943,2013,2013-07-23 00:00:00.000,3545195,"Roches, dalles",Vitesse < 5 cm/s,NaN
2,398944,2007,2007-08-29 00:00:00.000,3525909,NaN,NaN,NaN
3,398944,2007,2007-08-29 00:00:00.000,3525910,NaN,NaN,NaN
4,398944,2009,2009-09-28 00:00:00.000,3527276,NaN,NaN,NaN
...,...,...,...,...,...,...,...
55687,421014,2013,2013-06-27 00:00:00.000,3627210,NaN,NaN,NaN
55688,421014,2010,2010-06-09 00:00:00.000,3637746,NaN,NaN,NaN
55689,421014,2012,2012-06-21 00:00:00.000,3655503,NaN,NaN,NaN
55690,421014,2010,2010-06-09 00:00:00.000,3664704,NaN,NaN,NaN


In [36]:
liste_df_to_merge = liste_df.drop(columns=["date_prel", "facies_vitesse", "facies_profondeur", "facies_substrat"])
stn_lf = stations_df\
    .drop(columns=["nom_masse_eau", "st_y", "st_x", "date_year"])\
    .merge(liste_df_to_merge, left_on='id', right_on='id')
stn_taxon = stn_lf.merge(test_indiv, left_on='liste_faunistique_floristique', right_on='liste_ff_id').drop(columns=["liste_faunistique_floristique"])
stn_taxon.drop_duplicates(inplace=True)
stn_taxon.sort_values(["taxon_id", "liste_ff_id"])

,id,her,date_year,liste_ff_id,taxon_id,valeur
1,398944,22,2007,3551258,212,50.0
59,398944,22,2007,3551260,212,115.0
92,398944,22,2007,3551261,212,17.0
127,398944,22,2007,3551262,212,7.0
185,398944,22,2007,3551264,212,13.0
...,...,...,...,...,...,...
616865,410057,10,2013,3636540,933,380.0
616899,410057,10,2013,3636541,933,183.0
619050,410172,9,2013,3636575,933,5.0
619085,410172,9,2013,3636576,933,15.0


A vérifier, mais 1 relevé par stations par années

In [45]:
physicochimie_df.sort_values(["stn_id", "date_year", "parameter_name"])

,stn_id,date_year,valeur,parameter,parameter_name,unite,unite_name
52,398944,2007,0.064167,1335,Ammonium,162,milligramme par litre
111,398944,2007,94.350000,1374,Calcium,162,milligramme par litre
29,398944,2007,2.133333,1313,Demande Biochimique en oxygène en 5 jours (D.B...,175,milligramme d'oxygène par litre
35,398944,2007,23.583333,1314,Demande Chimique en Oxygène (D.C.O.),175,milligramme d'oxygène par litre
70,398944,2007,27.400000,1345,Dureté totale,28,degré français
...,...,...,...,...,...,...,...
187441,421014,2012,5.650000,1375,Sodium,162,milligramme par litre
187361,421014,2012,98.000000,1312,Taux de saturation en oxygène,243,pourcentage
187331,421014,2012,9.533333,1301,Température de l'Eau,27,degré Celsius
187402,421014,2012,0.000000,1346,Titre alcalimétrique (T.A.),28,degré français


In [44]:
physicochimie_to_merge = physicochimie_df.drop(columns=["date_year", "parameter_name", "unite_name"])
stn_taxon_pc = stn_taxon.merge(physicochimie_df, left_on=["id", "date_year"], right_on=["stn_id","date_year"])
stn_taxon_pc

,id,her,date_year,liste_ff_id,taxon_id,valeur_x,stn_id,valeur_y,parameter,parameter_name,unite,unite_name
0,398944,22,2007,3551258,906,120.0,398944,10.825000,1301,Température de l'Eau,27,degré Celsius
1,398944,22,2007,3551258,906,120.0,398944,7.764167,1302,Potentiel en Hydrogène (pH),264,unité pH
2,398944,22,2007,3551258,906,120.0,398944,14.250000,1305,Matières en suspension,162,milligramme par litre
3,398944,22,2007,3551258,906,120.0,398944,10.671667,1311,Oxygène dissous,175,milligramme d'oxygène par litre
4,398944,22,2007,3551258,906,120.0,398944,95.491667,1312,Taux de saturation en oxygène,243,pourcentage
...,...,...,...,...,...,...,...,...,...,...,...,...
2184295,421014,5,2012,3620253,364,80.0,421014,0.025000,1350,Phosphore total,162,milligramme par litre
2184296,421014,5,2012,3620253,364,80.0,421014,1.300000,1367,Potassium,162,milligramme par litre
2184297,421014,5,2012,3620253,364,80.0,421014,6.320000,1372,Magnésium,162,milligramme par litre
2184298,421014,5,2012,3620253,364,80.0,421014,91.900000,1374,Calcium,162,milligramme par litre
